# Basic imports

In [1]:
import pandas as pd
import os
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams.update({'figure.max_open_warning': 0})

In [2]:
current_path = os.getcwd()
root_path=current_path.replace('\\forward_feature_selection','')

# Loading dataframe

In [3]:
mixed_df=pd.read_csv(root_path+"\\molecules.csv",sep="\t")

# Functions

In [4]:
def save_df_to_disk(df,name:str,separator="\t"):
    df.to_csv(name,sep=separator,index = False, header=True)

In [5]:
def get_data_and_true_prediction(df,not_wanted_features:list):
        temp_df=df.drop(not_wanted_features,axis=1)
        y=temp_df[temp_df.columns[-1]]
        x=temp_df.drop([temp_df.columns[-1]],axis=1)
        
        return x,y   

In [6]:
def get_df_with_name_and_prediction(df,true_prediction,big_df):
    new_df=df
    new_df.insert(0,"m_name",big_df["m_name"].values)
    new_df=new_df.join(true_prediction)
    
    return new_df

# Forward selection

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [11]:
def get_roc_auc_score(x,y,model): # gets roc auc average
        cv_results = cross_validate(model, x, y, cv=10,scoring=('roc_auc'))
        roc_auc_avrg=cv_results['test_score'].mean()
        
        return roc_auc_avrg 

In [13]:
def forward_selection(x,y,model): # O(n) worst case scenario, where n depends on len(x.columns)
    
    def first_iteration(x,y,model):
        score_lst=[]
        for i in range(len(x.columns)):
            k=x.columns[i]
            temp_x=x[[k]]
            score=get_roc_auc_score(temp_x,y,model)
            score_lst.append(score)

        max_score = max(score_lst) # best score
        max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        new_features=[x.columns[i] for i in max_score_index]
        top_new_features=new_features[0]
        best_x=x[top_new_features]
        new_x=x.drop(top_new_features,axis=1)
        
        return best_x,new_x,max_score
    
    def else_iteration(best_x,x,y,model,actual_score):     
        new_x_lenght = len(x.columns)
        if (new_x_lenght > 0):
            score_lst=[]
            for i in range(new_x_lenght):
                k=x.columns[i]
                temp_x=x[[k]]
                temp_new_x=pd.concat([best_x,temp_x],axis=1, ignore_index=True)
                score=get_roc_auc_score(temp_new_x,y,model)
                score_lst.append(score)

            new_max_score = max(score_lst) # best score
            actual_best_score = actual_score # score passed from parameters

            if(new_max_score<actual_best_score):
                return best_x,actual_best_score # break condition, recursive function

            max_score_index=[i for i, j in enumerate(score_lst) if j == new_max_score] # indx with best score 

            new_features=[x.columns[i] for i in max_score_index]
            top_new_features=new_features[0]
            temp_x=x[top_new_features]
            best_x=pd.concat([best_x,temp_x],axis=1)
            new_x=x.drop(top_new_features,axis=1)

            return else_iteration(best_x,new_x,y,model,new_max_score)
        
        return best_x,actual_score
    
    best_x,new_x,score=first_iteration(x,y,model)
    best_x,score=else_iteration(best_x,new_x,y,model,score)
    
    return best_x,score

In [27]:
def SVM_forward_selection(x,y,c:list,gamma:list,krn='rbf',alfa=4):
    
    def get_best_score_index(score_lst):
        max_score = max(score_lst) # best score
        max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        
        return max_score_index[0]
    
    def inner_iteration(x,y,c,gamma:list):      
        print("c is now: {}".format(c))
        df_score_lst = []
        for i in gamma:
            SVC_clf = SVC(C=c, gamma=i,kernel=krn)
            df,score = forward_selection(x,y,SVC_clf)
            df_score_lst.append([df,score])
            
        score_lst = [y for [x,y] in df_score_lst]
        best_index = get_best_score_index(score_lst)
        
        print("Best inner model when gamma = {}".format(gamma[best_index]))
        print("\nFeatures\n")
        for c,i in enumerate(df_score_lst[best_index][0].columns):
            print("{}. {}".format(c+1,i))
        
        return df_score_lst[best_index][0],df_score_lst[best_index][1],gamma[best_index]
    
    outer_df_score_lst = []
    temp_best_score = 0
    
    tries = math.ceil((len(c)/alfa))+1 # spare tries if by some point the performance decreases
    
    for cnt,i in enumerate(c):
        best_inner_df,best_inner_score,best_gamma_value = inner_iteration(x,y,i,gamma)
        print("\nSpare tries: {}".format(tries))
        
        if((best_inner_score > temp_best_score) and tries > 0):
            temp_best_score = best_inner_score
            outer_df_score_lst.append([best_inner_df,best_inner_score,i,best_gamma_value])
            if (cnt > 0): print("This iteration had an improvement\n")
            elif (cnt == 0): print("")             
            tries = math.ceil((len(c)/alfa))+1             
        else:
            print("This iteration didn't have an improvement\n")
            tries = tries-1
        
        if (tries == 0):
            break
                                              
    outer_score_lst = [b for [a,b,c,d] in outer_df_score_lst]
    best_outer_index = get_best_score_index(outer_score_lst)
    
    print("Final results")
    print("Best model when c = {} ,gamma = {} ,roc_auc = {}%".format(outer_df_score_lst[best_outer_index][2],
                                                                 outer_df_score_lst[best_outer_index][3],
                                                                 outer_df_score_lst[best_outer_index][1]*100))
    
    return outer_df_score_lst[best_outer_index][0],outer_df_score_lst[best_outer_index][1]  

# Data

In [8]:
unnecesary_features=["m_name"]
x,y=get_data_and_true_prediction(mixed_df,unnecesary_features)

In [26]:
c_lst = [10e-2,10e-1,10e0,10e1] # > 0
gamma_lst = [10e-1,10e0,10e1] # > 1

# Comparison SVM

In [28]:
start = time.time()
df,score_normal = SVM_forward_selection(x,y,c_lst,gamma_lst)
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))

c is now: 0.1
Best inner model when gamma = 1.0

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_O
4. m_logp
5. n_aromatic_carbocycles
6. n_amide_bonds
7. fraction_CSP3

Spare tries: 2

c is now: 1.0
Best inner model when gamma = 1.0

Features

1. n_HBD
2. n_aliphatic_heterocycles
3. n_O
4. n_atoms_stereo_centers
5. n_amide_bonds
6. n_briged_head_atoms

Spare tries: 2
This iteration didn't have an improvement

c is now: 10.0
Best inner model when gamma = 1.0

Features

1. n_HBD
2. fraction_CSP3
3. n_non_strict_rotable_bonds
4. n_aromatic_carbocycles
5. n_hetero_cycles
6. n_O
7. n_aliphatic_rings
8. m_logp
9. n_amide_bonds
10. n_rings

Spare tries: 1
This iteration had an improvement

c is now: 100.0
Best inner model when gamma = 1.0

Features

1. n_HBD
2. fraction_CSP3
3. n_aliphatic_heterocycles
4. n_HBA

Spare tries: 2
This iteration didn't have an improvement

Final results
Best model when c = 10.0 ,gamma = 1.0 ,roc_auc = 85.45396195679105%


'00:16:18'